<a href="https://colab.research.google.com/github/farzanehnia/PYOMO/blob/main/Power_Generation_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
pip install -q pyomo

In [3]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd

In [26]:
#inputs
!rm inputs.xlsx

!wget --show-progress --continue -O /content/inputs.xlsx https://raw.github.com/farzanehnia/PYOMO/main/inputs.xlsx

dataGen = pd.read_excel('inputs.xlsx', sheet_name='gen')
dataLoad = pd.read_excel('inputs.xlsx', sheet_name='load')

Ng = len(dataGen)

import re
dataLoad['Load Demand'] = dataLoad['Load Demand'].astype('str').str.replace('\D', "").astype('float')
dataGen['Power Generation'] = dataGen['Power Generation'].astype('str').str.replace('\D', "").astype('float')


--2022-07-03 10:54:14--  https://raw.github.com/farzanehnia/PYOMO/main/inputs.xlsx
Resolving raw.github.com (raw.github.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.github.com (raw.github.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/farzanehnia/PYOMO/main/inputs.xlsx [following]
--2022-07-03 10:54:14--  https://raw.githubusercontent.com/farzanehnia/PYOMO/main/inputs.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9455 (9.2K) [application/octet-stream]
Saving to: ‘/content/inputs.xlsx’

/content/inputs.xls 100%[===================>]   9.23K  --.-KB/s    in 0s      

2022-07-03 10:54:15 (59.9 MB/s) - ‘/content/inputs.xls

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]


In [28]:

dataGen.head()
dataLoad['Load Demand'].loc[0]

50.0

In [31]:
#model

model = pyo.ConcreteModel()

model.Pg = pyo.Var(range(Ng), bounds=(0, None))
Pg = model.Pg

#constraints
Pg_sum = sum([Pg[g] for g in dataGen.id])
demand_sum = dataLoad['Load Demand'].sum()


model.balance = pyo.Constraint(expr = Pg_sum == demand_sum)

model.cond = pyo.Constraint(expr = Pg[0]+Pg[3] >= dataLoad['Load Demand'].iloc[0])

model.limits = pyo.ConstraintList()
for g in dataGen.id:
  model.limits.add(expr = Pg[g] <= dataGen['Power Generation'].iloc[g])

In [42]:
#Obj func

model.obj = pyo.Objective(expr = sum([Pg[g]*dataGen.Cost[g] for g in dataGen.id]))


    'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


In [43]:
#Cost sumation practice
print(sum([Pg[g]*dataGen['Cost'].loc[g] for g in dataGen.id]))
print(sum([Pg[g]*dataGen.Cost[g] for g in dataGen.id]))

0.1*Pg[0] + 0.05*Pg[1] + 0.3*Pg[2] + 0.4*Pg[3] + 0.01*Pg[4]
0.1*Pg[0] + 0.05*Pg[1] + 0.3*Pg[2] + 0.4*Pg[3] + 0.01*Pg[4]


In [45]:
!apt install glpk-utils
!pip install glpk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 49 not upgraded.
Need to get 692 kB of archives.
After this operation, 1,664 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsuitesparseconfig5 amd64 1:5.1.2-2 [9,044 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libamd2 amd64 1:5.1.2-2 [19.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 libcolamd2 amd64 1:5.1.2-2 [16.2 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 

In [55]:
opt = SolverFactory('glpk') #GLPK,cbc
results = opt.solve(model)

In [53]:
#print(results)
dataGen['Pg'] = [pyo.value(Pg[g]) for g in dataGen.id]
dataGen.head()

,id,Cost,Power Generation,Pg
0,0,0.10,20.0,20.0
1,1,0.05,10.0,10.0
2,2,0.30,40.0,35.0
3,3,0.40,50.0,30.0
4,4,0.01,5.0,5.0
